In [ ]:
!nvidia-smi

Sun May 30 16:36:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
from tensorflow.keras import layers
from keras.preprocessing import image
import tensorflow as tf 
from keras.models import Model,load_model
import os
import keras
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau, EarlyStopping
from keras.layers import Lambda
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import random
from PIL import Image 
from random import shuffle
%load_ext tensorboard
import datetime


**Prepare**

In [ ]:
import numpy as np
from numpy import genfromtxt
from numpy import asarray
import math
import copy
import os
from PIL import Image 
import cv2
import shutil

patch_size = 64 #input = 64x64
label_size = 128 #output = 128x128

#get RGGB bayer image
def bayer_reverse(img):
    height,width,c = img.shape;
    tmp = np.zeros([height,width]);
    for i in range( height ):
        for j in range( width ):
            if i % 2 == 0 :
                if j % 2 == 0:
                    tmp[i][j] = img[i][j][0];#R
                else:
                    tmp[i][j] = img[i][j][1];#G
            else :
                if j % 2 == 0:
                    tmp[i][j] = img[i][j][1];#G
                else:
                    tmp[i][j] = img[i][j][2];#B

    return tmp;

#split image to prepare the train set
def split(img,name,dir_path):
    height,width,c = img.shape;    
    count = 0;

    for i in range(0, height, 30):
        for j in range(0, width, 30):
            if( i + label_size < height and j + label_size < width ):                
                label = np.zeros([label_size,label_size,3])                
                tmp2  = np.zeros([label_size,label_size,3])                
                tmp2 = img[ i : i + label_size, j : j + label_size,:]
                label_img = Image.fromarray(tmp2)

                label[:,:,0] = tmp2[:,:,2]
                label[:,:,1] = tmp2[:,:,1]
                label[:,:,2] = tmp2[:,:,0]
                label_path = os.path.join(dir_path,'label/'+name.split('.')[0] +'_'+str(count)+'.png')                                                
                cv2.imwrite(label_path,label)

                
                zoom = label_img.resize((patch_size,patch_size), Image.BICUBIC)                 
                tmp3 = np.zeros([patch_size,patch_size])
                patch = np.zeros([patch_size,patch_size])

                zoom2 = np.array(zoom)                
                patch = bayer_reverse(zoom2)                
                                           
                patch_path = os.path.join(dir_path,'patch/'+name.split('.')[0] +'_'+str(count)+'.png')                          
                cv2.imwrite(patch_path, patch)

                count = count + 1    

def main():
    path = 'drive/MyDrive/Colab Notebooks/undergraduate_project'

    if os.path.exists(os.path.join(path,'patch')):
      shutil.rmtree(os.path.join(path,'patch'))
    os.makedirs(os.path.join(path,'patch'))

    if os.path.exists(os.path.join(path,'label')):
      shutil.rmtree(os.path.join(path,'label'))
    os.makedirs(os.path.join(path,'label'))


    dataset_path = os.path.join(path,'BSD200')
    entries = os.listdir(dataset_path)
    for entry in entries:
        print(entry)  
        img_path = dataset_path + '/' + entry
        img = Image.open(img_path)
        img = np.array(img)   
        split(img,entry,path)



if __name__ == '__main__':
    main()


254054.png
183055.png
56028.png
117054.png
245051.png
236017.png
376001.png
42078.png
368078.png
16052.png
15088.png
181018.png
159091.png
181091.png
97017.png
54005.png
311068.png
76002.png
105053.png
145053.png
153077.png
254033.png
106025.png
105019.png
376020.png
130034.png
41004.png
15004.png
274007.png
35091.png
80099.png
65019.png
113044.png
100098.png
202012.png
35010.png
112082.png
20008.png
169012.png
198004.png
249087.png
23080.png
108073.png
100080.png
46076.png
170054.png
109034.png
92059.png
161062.png
246016.png
239096.png
122048.png
187029.png
104022.png
27059.png
271008.png
138078.png
147062.png
239007.png
326038.png
227040.png
157036.png
55075.png
71046.png
310007.png
216066.png
216041.png
23084.png
8049.png
260081.png
299091.png
24063.png
61060.png
188063.png
94079.png
35008.png
268002.png
108041.png
365025.png
134052.png
232038.png
95006.png
385028.png
147021.png
181079.png
55067.png
365073.png
246053.png
242078.png
151087.png
43083.png
156079.png
35058.png
189003.p

**Training**

In [ ]:
import numpy as np
from tensorflow.keras import layers
from keras.preprocessing import image
import tensorflow as tf 
from keras.models import Model,load_model
import os
import keras
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau, EarlyStopping
from keras.layers import Lambda
from keras.optimizers import Adam
from keras import backend as K
import random
from PIL import Image 
from random import shuffle

train_image = []
train_label = []

dir_path = 'drive/MyDrive/Colab Notebooks/undergraduate_project'
patch_path = os.path.join(dir_path,'patch')
entries = os.listdir(patch_path)
for entry in entries:  
  im = Image.open(patch_path+'/'+entry)
  img = image.img_to_array(im)
  # print(img.shape)
  # img = img/255.
  train_image.append(img)
train_image= np.stack(train_image)

print(train_image.shape)
  

label_path = os.path.join(dir_path,'label')
entries = os.listdir(label_path)
for entry in entries:  
  im = Image.open(label_path+'/'+entry)
  img = image.img_to_array(im)
  # img = img/255.
  train_label.append(img)
train_label = np.stack(train_label)

print(train_label.shape)



index = [i for i in range(train_image.shape[0])]
shuffle(index)
train_image = train_image[index,:,:,:];
train_label = train_label[index,:,:,:];

# np.save('drive/My Drive/Colab Notebooks/undergraduate_project/train_image.npy', train_image)
# np.save('drive/My Drive/Colab Notebooks/undergraduate_project/train_label.npy', train_label)

(16800, 64, 64, 1)
(16800, 128, 128, 3)


In [ ]:
############################# Model Structure ################################################
def create_model():
  inputs = keras.Input(shape=(None,None,1))

  ini = keras.layers.Conv2D(filters = 128, #feature map number
                     kernel_size = 3, 
                     strides = 1,  # 2                     
                     padding = 'same', 
                     input_shape = (None,None,1))(inputs)

  x = keras.layers.PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(ini)
  
  ini = keras.layers.Conv2D(filters = 128, #feature map number
                     kernel_size = 3, 
                     strides = 1,  # 2
                     activation = 'relu',
                     padding = 'same', 
                     input_shape = (None,None,128))(x)

  x = keras.layers.PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(ini)

  ini = keras.layers.Conv2D(filters = 64, #feature map number
                     kernel_size = 3, 
                     strides = 1,  # 2
                     activation = 'relu',
                     padding = 'same', 
                     input_shape = (None,None,128))(x)

  x = keras.layers.PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(ini)

  ##Subpixel Construction
  sub_layer_2 = Lambda(lambda x:tf.nn.space_to_depth(x,2)) 
  init = sub_layer_2(inputs=x)


  ##Learning Residual (DCNN)
  ####Conv 3x3x64x64 + PReLu
  x = keras.layers.Conv2D(filters = 64, #feature map number
                     kernel_size = 3, 
                     strides = 1,  # 2
                     padding = 'same', 
                     activation = 'relu',
                     input_shape = (None,None,1))(init)
  x = keras.layers.BatchNormalization()(x)                     
  x = keras.layers.Conv2D(filters = 64, #feature map number
                     kernel_size = 3, 
                     strides = 1,  # 2
                     padding = 'same', 
                     activation = 'relu',
                     input_shape = (None,None,64))(x)
  x = keras.layers.BatchNormalization()(x)                     
  x = keras.layers.Conv2D(filters = 64, #feature map number
                     kernel_size = 3, 
                     strides = 1,  # 2
                     padding = 'same', 
                     activation = 'relu',
                     input_shape = (None,None,64))(x)
  x = keras.layers.BatchNormalization()(x)                     
  # x = keras.layers.MaxPooling2D((2,2),padding='same')(x)
  # x = keras.layers.BatchNormalization()(x)  
  # x = keras.layers.PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(x)

  ####Residual Block
  for i in range(6):
    start = x
    Conv1 = keras.layers.Conv2D(filters=64,
                        kernel_size = 1, 
                        strides = 1,  # 2
                        padding = 'same',                                             
                        input_shape = (None,None,64))(start)
    Conv1 = keras.layers.LeakyReLU()(Conv1)
    # Conv1 = keras.layers.MaxPooling2D((2,2),padding='same')(Conv1)
    # Conv1_BN = keras.layers.BatchNormalization()(Conv1)
    # Conv1_BN = Dropout(0.5)(Conv1_BN)
    
    # PReLu = keras.layers.PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(Conv1)
    Conv2 = keras.layers.Conv2D(filters=64,
                        kernel_size = 3, 
                        strides = 1,  # 2
                        padding = 'same',                                                
                        input_shape = (None,None,64))(Conv1)
    Conv2 = keras.layers.LeakyReLU()(Conv2)
    # Conv2 = keras.layers.MaxPooling2D((2,2),padding='same')(Conv2)
    # Conv2_BN = keras.layers.BatchNormalization()(Conv2)
    # Conv2_BN = Dropout(0.5)(Conv2_BN)                        
    # PReLu = keras.layers.PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(Conv2)
    Conv3 = keras.layers.Conv2D(filters=64,
                        kernel_size = 1, 
                        strides = 1,  # 2
                        padding = 'same',         
                        input_shape = (None,None,64))(Conv2)
    # Conv3 = keras.layers.MaxPooling2D((2,2),padding='same')(Conv3)
    # Conv3 = keras.layers.LeakyReLU()(Conv3)
    
    # Concatenate        
    x = keras.layers.Add()([Conv3,x])
    
  # x = keras.layers.MaxPooling2D((2,2),padding='same')(x)
  ####Conv 3x3x64x64 + PReLu
  x = keras.layers.Conv2D(filters = 64,
                     kernel_size = 3, 
                     strides = 1,  # 2
                     padding = 'same',
                     activation = 'relu',
                     input_shape = (None,None,64))(x)
  x = keras.layers.Conv2D(filters = 64,
                     kernel_size = 3, 
                     strides = 1,  # 2
                     padding = 'same',
                     activation = 'relu',
                     input_shape = (None,None,64))(x)
  x = keras.layers.Conv2D(filters = 64,
                     kernel_size = 3, 
                     strides = 1,  # 2
                     padding = 'same',
                     activation = 'relu',
                     input_shape = (None,None,64))(x)            
  # x = keras.layers.MaxPooling2D((2,2),padding='same')(x)
  # x = keras.layers.BatchNormalization()(x)                                                                 
  # x = keras.layers.PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(x)


  ####Conv 3x3x64x48
  x = keras.layers.Conv2D(filters = 64,
                     kernel_size = 3, 
                     strides = 1,  
                     padding = 'same',
                     activation = 'relu',                   
                     input_shape = (None,None,64))(x)
  x = keras.layers.Conv2D(filters = 64,
                     kernel_size = 3, 
                     strides = 1,  
                     padding = 'same',
                     activation = 'relu',                   
                     input_shape = (None,None,64))(x)
  x = keras.layers.Conv2D(filters = 48,
                     kernel_size = 3, 
                     strides = 1,  
                     padding = 'same',
                     activation = 'relu',                   
                     input_shape = (None,None,64))(x)
  # x = keras.layers.MaxPooling2D((2,2),padding='same')(x)                     
  # x = keras.layers.BatchNormalization()(x)

  ###########Learning Residual (DCNN)############
  

  ##Recovery From Subpixel
  sub_layer = Lambda(lambda x:tf.nn.depth_to_space(x,4)) 
  Residual_Output = sub_layer(inputs=x)
  

  ##Initial Prediction
  R = Lambda(lambda x: x[:,:,:,0])(init)
  G = Lambda(lambda x: x[:,:,:,1:3])(init)
  G = Lambda(lambda x: K.mean(x, axis=3))(G)
  B = Lambda(lambda x: x[:,:,:,3])(init)
  # print(init.shape)
  # print(R.shape)
  # print(G.shape)
  # print(B.shape)
  R = Lambda(lambda x: tf.expand_dims(x, -1))(R)
  G = Lambda(lambda x: tf.expand_dims(x, -1))(G)
  B = Lambda(lambda x: tf.expand_dims(x, -1))(B)
  
  #rgb = tf.keras.backend.stack((R, G,B),axis =  3)
  # print(R.shape)
  rg = keras.layers.Concatenate(axis = 3)([R , G])
  rgb = keras.layers.Concatenate(axis = 3)([rg,B])
  # print(rgb.shape)
  Coarse_Output = keras.layers.UpSampling2D(size=(4, 4), interpolation="bilinear")(rgb) #size=4 , from W/2,H/2 ---> 2W,2H
  # Coarse_Output = keras.layers.MaxPooling2D((2,2),padding='same')(Coarse_Output)


  ## + 
  outputs = keras.layers.Add()([Residual_Output,Coarse_Output])
  #outputs = Residual_Output
  model = keras.Model(inputs=inputs, outputs=outputs, name="JDMSR_model")  
  return model

model = create_model()
model.summary()

Model: "JDMSR_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 1 1280        input_1[0][0]                    
__________________________________________________________________________________________________
p_re_lu (PReLU)                 (None, None, None, 1 128         conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 1 147584      p_re_lu[0][0]                    
________________________________________________________________________________________

In [ ]:
batch_size = 16
lr = 0.001
e_num = 50
dir_path = 'drive/My Drive/Colab Notebooks/undergraduate_project'

In [ ]:
model = create_model()
model.summary()

sgd = SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=1.0)

model.compile(optimizer=Adam(), loss = 'mean_squared_error', metrics = ['mse'])

checkpoint = ModelCheckpoint(os.path.join(dir_path,'model.hdf5'),verbose=1, monitor='loss', save_best_only=True, save_weights_only=True)
rrp = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1, mode='min', min_lr=0.000002)

early_stopping = EarlyStopping(monitor='loss', patience=10, verbose=1, mode='auto')

print(train_image.shape)
print(train_label.shape)
history = model.fit(train_image, train_label, epochs=e_num, batch_size=batch_size,verbose=1,validation_split = 0.1,callbacks=[checkpoint, rrp],shuffle = True)

Model: "JDMSR_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_30 (Conv2D)              (None, None, None, 1 1280        input_2[0][0]                    
__________________________________________________________________________________________________
p_re_lu_3 (PReLU)               (None, None, None, 1 128         conv2d_30[0][0]                  
__________________________________________________________________________________________________
conv2d_31 (Conv2D)              (None, None, None, 1 147584      p_re_lu_3[0][0]                  
________________________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/50
945/945 [==============================] - 102s 56ms/step - loss: 1067.0114 - mse: 1067.0114 - val_loss: 199.6852 - val_mse: 199.6852

Epoch 00001: loss improved from inf to 411.73077, saving model to drive/My Drive/Colab Notebooks/undergraduate_project/model.hdf5
Epoch 2/50
945/945 [==============================] - 53s 56ms/step - loss: 195.0287 - mse: 195.0287 - val_loss: 163.8533 - val_mse: 163.8533

Epoch 00002: loss improved from 411.73077 to 185.95012, saving model to drive/My Drive/Colab Notebooks/undergraduate_project/model.hdf5
Epoch 3/50
945/945 [==============================] - 53s 56ms/step - loss: 175.8356 - mse: 175.8356 - val_loss: 147.4516 - val_mse: 147.4516

Epoch 00003: loss improved from 185.95012 to 166.49963, saving model to drive/My Drive/Colab Notebooks/undergraduate_project/model.hdf5
Epoch 4/50
945/945 [==============================] - 53s 56ms/step - loss: 154.6457 - mse: 154.6457 - val_loss: 137.7859 - val_mse: 137.7859

Epoch 00004: loss impro

In [ ]:
# Plotting
import matplotlib.pyplot as plt
fig = plt.figure()
plt.plot()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Accuracy - 2')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')
plt.savefig('model_accuracy2.png')
plt.show()

plt.plot()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss - 2')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.savefig('model_loss2.png')
plt.show()

**Validation**

In [ ]:
from keras.models import load_model
from keras.layers import Lambda
from keras.preprocessing import image
import keras
import tensorflow as tf 
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image 
from keras import backend as K
import os
import math
import shutil

dir_path = 'drive/My Drive/Colab Notebooks/undergraduate_project'
model = create_model()
model.load_weights(os.path.join(dir_path,'model.hdf5'))

input_path = os.path.join(dir_path, 'koda')
output_path = os.path.join(dir_path, 'output')
output_path2 = os.path.join(dir_path, 'output2')
output_path3 = os.path.join(dir_path, 'output3')

if os.path.exists(output_path):
    shutil.rmtree(output_path)
os.makedirs(output_path)

if os.path.exists(output_path2):
    shutil.rmtree(output_path2)
os.makedirs(output_path2)

if os.path.exists(output_path3):
    shutil.rmtree(output_path3)
os.makedirs(output_path3)

entries = os.listdir(input_path)

for entry in entries:
    # Test Image
    print('-----------')
    path = input_path+'/'+entry
    test_image = Image.open(path)
    print(test_image.size)
    
    if (test_image.size[0]/2)%2 != 0: #odd size
      test_image = test_image.resize((test_image.size[0]-1, test_image.size[1]), Image.BICUBIC)
    if (test_image.size[1]/2)%2 != 0:
      test_image = test_image.resize((test_image.size[0], test_image.size[1]-1), Image.BICUBIC)
    path = output_path+'/'+entry
    test_image.save(path)
    print(test_image.size)

    test_image = test_image.resize((test_image.size[0]//2, test_image.size[1]//2), Image.BICUBIC)
    print(test_image.size)
    test_image_array = np.array(test_image)
    test_image_array = bayer_reverse(test_image_array)
    test_image_array = test_image_array[:,:,np.newaxis]
    test_image = image.array_to_img(test_image_array)
    test_image.save(output_path3+'/'+entry)
    # print(test_image.shape)
    print(test_image.size)
    
    
    # print(test_image.shape)
    test_image = np.array(test_image)
    test_image = test_image[np.newaxis,:,:]
    try:
      out = model.predict(test_image)
      out = out[0]
      out = image.array_to_img(out)
      path = output_path2+'/'+entry
      out.save(path)

    except:
      path = input_path+'/'+entry
      test_image = Image.open(path)
      if (test_image.size[0]//scale)%2 != 0:
        test_image = test_image.resize(((test_image.size[0]//scale) - 1, test_image.size[1]//scale), Image.BICUBIC)
      if (test_image.size[1]//scale)%2 != 0:
        test_image = test_image.resize((test_image.size[0]//scale , (test_image.size[1]//scale) - 1), Image.BICUBIC)
      test_image_array = np.array(test_image)
      test_image_array = bayer_reverse(test_image_array)
      test_image_array = test_image_array[:,:,np.newaxis]
      test_image = image.array_to_img(test_image_array)
      print(test_image.size)
      test_image.save(output_path3+'/'+entry)
      test_image = np.array(test_image)
      test_image = test_image[np.newaxis,:,:]
      out = model.predict(test_image)
      out = out[0]
      out = image.array_to_img(out)
      path = output_path2+'/'+entry
      out.save(path)


    # print(out.shape)
    

# print(len(entries))

-----------
(481, 321)
(480, 320)
(240, 160)
(240, 160)
-----------
(481, 321)
(480, 320)
(240, 160)
(240, 160)
-----------
(481, 321)
(480, 320)
(240, 160)
(240, 160)
-----------
(481, 321)
(480, 320)
(240, 160)
(240, 160)
-----------
(481, 321)
(480, 320)
(240, 160)
(240, 160)
-----------
(481, 321)
(480, 320)
(240, 160)
(240, 160)
-----------
(481, 321)
(480, 320)
(240, 160)
(240, 160)
-----------
(321, 481)
(320, 480)
(160, 240)
(160, 240)
-----------
(481, 321)
(480, 320)
(240, 160)
(240, 160)
-----------
(481, 321)
(480, 320)
(240, 160)
(240, 160)
-----------
(481, 321)
(480, 320)
(240, 160)
(240, 160)
-----------
(481, 321)
(480, 320)
(240, 160)
(240, 160)
-----------
(481, 321)
(480, 320)
(240, 160)
(240, 160)
-----------
(481, 321)
(480, 320)
(240, 160)
(240, 160)
-----------
(481, 321)
(480, 320)
(240, 160)
(240, 160)
-----------
(321, 481)
(320, 480)
(160, 240)
(160, 240)
-----------
(481, 321)
(480, 320)
(240, 160)
(240, 160)
-----------
(481, 321)
(480, 320)
(240, 160)
(24

**Performance**

In [ ]:
import cv2
import numpy as np
import math
import os
from google.colab.patches import cv2_imshow
from skimage.measure import compare_ssim
from skimage.measure import compare_ssim as ssim

def calculate_psnr(original, contrast):
    mse = np.mean((original - contrast) ** 2)
    if mse == 0:
        return 100
    max_value = 255.0
    return 10 * math.log10(max_value**2 / mse)    
    

path = 'drive/My Drive/Colab Notebooks/undergraduate_project'
input_path = os.path.join(path,'output')
output_path = os.path.join(path,'output2')
entries = os.listdir(input_path)
count = 0
total_psnr = 0.
total_ssim = 0.

for entry in entries:  
  img1 = cv2.imread(os.path.join(input_path,entry))
  img2 = cv2.imread(os.path.join(output_path,entry))
  img1 = cv2.resize(img1, (img2.shape[1], img2.shape[0]), interpolation=cv2.INTER_CUBIC)
    


  # PSNR
  psnr = calculate_psnr(img1,img2)
  print("PSNR-{0}: {1:.10f}dB".format(entry,psnr))

  # SSIM
  ssim = compare_ssim(img1, img2, data_range=255.0 - 0.0,multichannel=True)
  print("SSIM-{0}: {1:.10f}".format(entry,ssim))

  total_psnr += psnr
  total_ssim += ssim
  count += 1
print(count)

total_psnr = total_psnr / count
total_ssim = total_ssim / count
print("\n=====================================")
print("Average PSNR:{:.10f}".format(total_psnr))
print("Average SSIM:{:.10f}".format(total_ssim))




PSNR-126007.jpg: 27.5680608067dB
SSIM-126007.jpg: 0.8073178361
PSNR-24077.jpg: 27.8912181256dB
SSIM-24077.jpg: 0.8526484924
PSNR-157055.jpg: 27.3065674168dB
SSIM-157055.jpg: 0.8073012958
PSNR-103070.jpg: 27.2596979936dB
SSIM-103070.jpg: 0.8832143149


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.


PSNR-156065.jpg: 30.3648811152dB
SSIM-156065.jpg: 0.7835894835
PSNR-170057.jpg: 29.1322365640dB
SSIM-170057.jpg: 0.9146643532
PSNR-76053.jpg: 27.5901472968dB
SSIM-76053.jpg: 0.8072015272
PSNR-78004.jpg: 28.7543717853dB
SSIM-78004.jpg: 0.8607183036
PSNR-38092.jpg: 28.6167747435dB
SSIM-38092.jpg: 0.8504220508
PSNR-253027.jpg: 27.2006672528dB
SSIM-253027.jpg: 0.7209558015
PSNR-16077.jpg: 27.3801107935dB
SSIM-16077.jpg: 0.7811798712
PSNR-163085.jpg: 30.6100132108dB
SSIM-163085.jpg: 0.9133935782
PSNR-219090.jpg: 28.7127667467dB
SSIM-219090.jpg: 0.8668010378
PSNR-58060.jpg: 29.7846089887dB
SSIM-58060.jpg: 0.7441327902
PSNR-296007.jpg: 29.1679615768dB
SSIM-296007.jpg: 0.8839202206
PSNR-101085.jpg: 29.0546724208dB
SSIM-101085.jpg: 0.7822121406
PSNR-119082.jpg: 28.7310037200dB
SSIM-119082.jpg: 0.8489635208
PSNR-300091.jpg: 27.9209532266dB
SSIM-300091.jpg: 0.8576244869
PSNR-167062.jpg: 31.9829932324dB
SSIM-167062.jpg: 0.9022111714
PSNR-45096.jpg: 35.9661096598dB
SSIM-45096.jpg: 0.9433544105
PSNR